In order to run this notebook you have to install the package `trump` in your computer. In the root folder of `trump` package you can run:
```python
pip3 install -e .
```
to install as a developer mode or
```python
pip3 install .
```
If you want, you can also bootstrap the virtual environment and run your notebook inside the virtual environment.
```bash
bash bootstrap-python-env.sh
source trump-env/bin/activate
jupyter notebook
```
This package was developed to make feasible read the code in the notebook, write tests for the functions and also to have a more manageable code base.

In [8]:
from trump.load import load_tweets
from trump import treat

from IPython.display import HTML

# Demystifying POTUS, one tweet at a time...

Twitter plays a crucial role in politics these days. Gone are the days of door-to-door campaigning and trying to reach the last man. Today, power is weilded by those who can tweet. A carefully worded 140 character phrase carries the ability to swing states and potentially change the course of an entire nation. The goal of our project is to analyse the impact of Twitter on society and understand how ideas are spread across a network.

In order to investigate this question, our proposal is to analyse the tweets posted by Trump on Twitter over time to discover what was the main focus of his campaign, how it changed and how the society reacts to his discourses and proposals.

The Trump Twitter Archive is a project that has been collecting Trump's tweets since 2009 but the main focus of this project will be the period of his campaign.

**For the milestone we have the following topics to be addressed:**

- Choose the word scores dataset based on the amount of words available and context (what is the context upon which the scores were built).
- Discover the window time feasible to develop the project in a machine with 8GB of RAM. The period of campaign must be included in this interval.
- Descriptive and exploratory analysis of tweets' text.
- Clean stop words from dataset.
- Check if LDA model is a feasible approach to find topics and then classify tweets.
- Have a final proposal of the topics that will be analysed to answer the `Research Questions`

That will be answered in the next sections.

## Topics - Milestone 2

### Dataset partition
- Choose the word scores dataset based on the amount of words available and context (what is the context upon which the scores were built).
- Discover the window time feasible to develop the project in a machine with 8GB of RAM. The period of campaign must be included in this interval.

In [ ]:
# TODO

In [ ]:
df = load_tweets()
HTML(df.head().to_html())

### Descriptive Analysis
Descriptive and exploratory analysis of tweets' text.

In [ ]:
# TODO

### Clean text
Clean stop words from dataset.

In [ ]:
# TODO

### Topics classification
Check if LDA model is a feasible approach to find topics and then classify tweets.

In [ ]:
# TODO

### Research Questions
Have a final proposal of the topics that will be analysed to answer the `Research Questions`

In [13]:
# TODO